To Do:
- create downloadable csv for each jurisdiction
- deal with duble field entries esp from the simple normalizing

In [1]:
import pandas as pd
import urllib.request, json 
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import PyPDF2

import pikepdf

import os
from os import walk
import os.path
from os import path
import numpy as np
#!pip install py-readability-metrics
from readability import Readability
#!python -m nltk.downloader punkt

import time
from datetime import date

import spacy
from sklearn.preprocessing import normalize

from joblib import dump, load


import networkx as nx

import math

from sklearn.metrics.pairwise import cosine_similarity# define matrix with all zero values

#import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
nlp = spacy.load('en_core_web_lg')

In [3]:
def vectorize(text):
    output = nlp(str(text)).vector   
    return output

def norm(row):
    try:
        matrix = row.reshape(1,-1).astype(np.float64)
        return normalize(matrix, axis=1, norm='l1')[0]
    except Exception as e: 
        print("===================")
        print(row)
        print("-------------------")
        print(e)
        print("===================")
        #return np.NaN

In [4]:
included_fields = load('../data/processed/ML/norm_fields/included_fields.joblib') 
jurisdictions = load('../data/processed/ML/norm_fields/jurisdictions.joblib') 
groups = load('../data/processed/ML/norm_fields/groups.joblib') 
clf_field_names = load('../data/processed/ML/norm_fields/clf_field_names.joblib') 

stop_words = set(stopwords.words('english'))

def reformat_field(text,max_length=30):
    # h/t https://towardsdatascience.com/nlp-building-a-summariser-68e0c19e3a93
    
    #print(text)
    
    orig_title = text.lower()
    orig_title = re.sub("[^a-zA-Z]+"," ",orig_title)
    orig_title_words = orig_title.split()
   
    deduped_sentence = []
    for word in orig_title_words:
        if word not in deduped_sentence:
            deduped_sentence.append(word)
            
    filtered_sentence = [w for w in deduped_sentence if not w.lower() in stop_words]   

    filtered_title_words = filtered_sentence
    
    characters = len(' '.join(filtered_title_words))
    
    if characters > 0:

        words = len(filtered_title_words)
        av_word_len = math.ceil(len(' '.join(filtered_title_words))/len(filtered_title_words))
        x_words = math.floor((max_length)/av_word_len)


        sim_mat = np.zeros([len(filtered_title_words),len(filtered_title_words)])# will populate it with cosine_similarity values 
        # for each word compared to other
        for i in range(len(filtered_title_words)):
            for j in range(len(filtered_title_words)):
                if i != j:
                    sim_mat[i][j] = cosine_similarity(nlp(filtered_title_words[i]).vector.reshape(1,300), nlp(filtered_title_words[j]).vector.reshape(1,300))[0,0]

        try:
            nx_graph = nx.from_numpy_array(sim_mat)
            scores = nx.pagerank(nx_graph)# print final values of words
            sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)

            if x_words > len(scores):
                x_words=len(scores)

            i = 0
            new_title = ""
            for x in filtered_title_words:
                #print(scores[i],sorted_scores[x_words][1])
                if scores[i] >= sorted_scores[x_words-1][1]: 
                    if len(new_title)>0: new_title+="_"
                    new_title += x
                i+=1

            return new_title
        except:
            return '_'.join(filtered_title_words)
    else:
        return re.sub("\s+","_",text.lower())
    

def normalize_name(jur,group,n,per,last_field,this_field):
    
    # Add hard coded conversions maybe by calling a function
    # if returns 0 then fail over to ML or otherway around poor prob -> check hard-coded

    if this_field not in included_fields:
        this_field = reCase(this_field)

        out_put = re.sub("^(My\s)?Name$","users1_name",this_field, flags=re.IGNORECASE)
        out_put = re.sub("^Printed Name\s?\d*$","users1_name",out_put, flags=re.IGNORECASE)  
        out_put = re.sub("^(DOB|Date of Birth|Birthday)$","users1_birthdate",out_put, flags=re.IGNORECASE)

        out_put = re.sub("^(Street )?Address$","users1_address_line_one",out_put, flags=re.IGNORECASE)
        out_put = re.sub("^City State Zip$","users1_address_line_two",out_put, flags=re.IGNORECASE)
        out_put = re.sub("^City$","users1_address_city",out_put, flags=re.IGNORECASE)
        out_put = re.sub("^Zip( Code)?$","users1_address_zip",out_put, flags=re.IGNORECASE)

        out_put = re.sub("^Phone$","users1_phone_number",out_put, flags=re.IGNORECASE)
        out_put = re.sub("^Email$","users1_email",out_put, flags=re.IGNORECASE)

        out_put = re.sub("^(Court\s)?Case\s?(No|Number)?\s?A?$","docket_number",out_put, flags=re.IGNORECASE)
        out_put = re.sub("^File\s?(No|Number)?\s?A?$","docket_number",out_put, flags=re.IGNORECASE)

        out_put = re.sub("^Date\s?\d*$","signature_date",out_put, flags=re.IGNORECASE)

        conf = 1.0

        if out_put==this_field:
            params = []
            for item in jurisdictions:
                if jur== item:
                    params.append(1)
                else:
                    params.append(0)
            for item in groups:
                if group== item:
                    params.append(1)
                else:
                    params.append(0)
            params.append(n)
            params.append(per)
            for vec in norm(vectorize(this_field)):
                params.append(vec)
            #for vec in norm(vectorize(last_field)):
            #    params.append(vec)

            for item in included_fields:
                if last_field==item:
                    params.append(1)
                else:
                    params.append(0)

            pred = clf_field_names.predict([params])
            prob = clf_field_names.predict_proba([params])

            conf = prob[0].tolist()[prob[0].tolist().index(max(prob[0].tolist()))]
            out_put = pred[0]
            
    else:
        out_put = this_field
        conf = 1
            
    if out_put in included_fields:
        if conf >= 0:
            return "*"+out_put,conf #+"| was <i>%s</i> (%.2f conf)"%(this_field,conf) #, conf
        else:
            return reformat_field(this_field),conf #+"| was <i>%s</i> (%.2f conf)"%(this_field,conf) #, conf
    else:
        return reformat_field(this_field),conf #+"| was <i>%s</i> (%.2f conf)"%(this_field,conf) #, conf


In [5]:
def reCase(text):
    output = re.sub("(\w|\d)(_|-)(\w|\d)","\\1 \\3",text.strip())
    output = re.sub("([a-z])([A-Z]|\d)","\\1 \\2",output)
    output = re.sub("(\d)([A-Z]|[a-z])","\\1 \\2",output)
    return output

In [6]:
def read_pdf (file):
    try:
        pdfFile = PyPDF2.PdfFileReader(open(file, "rb"))
        if pdfFile.isEncrypted:
            try:
                pdfFile.decrypt('')
                #print ('File Decrypted (PyPDF2)')
            except:
                #
                #
                # This didn't go so well on my Windows box so I just ran this in the pdf folder's cmd:
                # for %f in (*.*) do copy %f temp.pdf /Y && "C:\Program Files (x86)\qpdf-8.0.2\bin\qpdf.exe" --password="" --decrypt temp.pdf %f
                #
                #
                #
                
                command="cp "+file+" tmp/temp.pdf; qpdf --password='' --decrypt tmp/temp.pdf "+file
                os.system(command)
                #print ('File Decrypted (qpdf)')
                #re-open the decrypted file
                pdfFile = PyPDF2.PdfFileReader(open(file, "rb"))
        text = ""
        for page in pdfFile.pages:
            text = text + " " + page.extractText()
        return text
    except:
        return ""


In [7]:
def parse_form(fileloc,title=None,jur=None,cat=None,normalize=1):
    f = PyPDF2.PdfFileReader(fileloc)
    npages = f.getNumPages()
    ff = f.getFields()
    if ff:
        fields = list(ff.keys())
    else:
        fields = []
    f_per_page = len(fields)/npages
    text = read_pdf(fileloc)
    try:
        readbility = int(Readability(text).flesch_kincaid().grade_level)
    except:
        readbility = None
    
    if title is None:
        title = reCase(re.search("(.*)\n",text).group(1).strip())
        
    if normalize==1:
        i = 0 
        length = len(fields)
        last = "null"
        new_fields = []
        new_fields_conf = []
        for field in fields:
            #print(jur,cat,i,i/length,last,field)
            this_field,this_conf = normalize_name(jur,cat,i,i/length,last,field)
            new_fields.append(this_field)
            new_fields_conf.append(this_conf)
            last = field
        
        new_fields = [v + "__" + str(new_fields[:i].count(v) + 1) if new_fields.count(v) > 1 else v for i, v in enumerate(new_fields)]
    else:
        new_fields = fields
    
    stats = {
            "title":title,
            "category":cat,
            "pages":npages,
            "reading grade level": readbility,
            "avg fields per page": f_per_page,
            "fields":new_fields,
            "fields_conf":new_fields_conf,
            "fields_old":fields
            }    
    
    my_pdf = pikepdf.Pdf.open(fileloc)
    fields_too = my_pdf.Root.AcroForm.Fields
    k =0
    for field in new_fields:
        fields_too[k].T = re.sub("^\*","",field)
        k+=1
        
    #f2.T = 'new_hospital_name'
    filename = re.search("\/(\w*\.pdf)$",fileloc).groups()[0]
    my_pdf.save('../data/processed/forms/%s'%(filename))
    
    return stats

In [ ]:
#parse_form("../data/processed/www.utcourts.gov/forms/898269a99ff1c65be10b1ae35bb34ba469fc14b7301b7ed7b126d195.pdf",title=None,jur="UT",cat=None,normalize=1)
#parse_form("../data/processed/www.utcourts.gov/forms/2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065cd27bde8c7.pdf",title=None,jur="UT",cat=None,normalize=1)
parse_form("../data/processed/www.utcourts.gov/forms/6ec7576210513907e699b5adf3397639507c688801a60bc34c201984.pdf",title=None,jur="UT",cat=None,normalize=1)

In [9]:
files_df_ut = pd.read_csv("../data/raw/www.utcourts.gov/form_data.csv")
files_df_mi = pd.read_csv("../data/raw/www.courts.michigan.gov/form_data.csv")

df = pd.concat([files_df_ut,files_df_mi],ignore_index=True)

In [10]:
df["pages"] = ""
df["fields"] = ""
df["fields_conf"] = ""
df["fields_old"] = ""
df["f_per_p"] = ""
df["reading"] = ""

for index,row in df.iterrows():
    if row["pages"] == "":
        try:
            stats = parse_form("../data/processed/"+row["source"]+"/forms/"+row["id"]+".pdf",row["title"],row["jurisdiction"],row["group"],1)
            df.at[index, 'pages'] = stats["pages"]
            df.at[index, 'fields'] = stats["fields"]
            df.at[index, 'fields_conf'] = stats["fields_conf"]
            df.at[index, 'fields_old'] = stats["fields_old"]
            df.at[index, 'f_per_p'] = stats["avg fields per page"]
            df.at[index, 'reading'] = stats["reading grade level"]
            #print(index)
        except:
            print("error: "+"../data/raw/"+row["source"]+"/forms/"+row["id"]+".pdf")
        
df

error: ../data/raw/www.utcourts.gov/forms/e6124de37c9e4ef524d46b368c582ac2adf999778566d68b1a3dd820.pdf
error: ../data/raw/www.utcourts.gov/forms/032c7267803815f129aae6465e8536fcc80dedb75360cf50dff3baeb.pdf


error: ../data/raw/www.courts.michigan.gov/forms/531b8bf75814ce2a9757fcae1b1926bacce15ab816f721165bf22b85.pdf


error: ../data/raw/www.courts.michigan.gov/forms/bfa644ad53e07138e85ce99e4eb01ed784805b9bd6fd08844623a4e8.pdf


error: ../data/raw/www.courts.michigan.gov/forms/6b79c1e8fda21d9a395e262f564c03a15af2761add0e4273f879d477.pdf
error: ../data/raw/www.courts.michigan.gov/forms/f9b72d647368a16b4abd3ffe22c2c7fa0204d4cdd1d1f19a110f00a1.pdf


error: ../data/raw/www.courts.michigan.gov/forms/bba53d81e3ffa8d9769cacb36b726b1c5c4888aba26576cb5fd16c31.pdf


...

error: ../data/raw/www.courts.michigan.gov/forms/2e28ba148c0a7321285d2fb5d58dc578c0d861c2b12d845f0cace96a.pdf
error: ../data/raw/www.courts.michigan.gov/forms/c99b9eecb27c62af29862987aa3da60254f9c221d88bc486d83518a3.pdf
error: ../data/raw/www.courts.michigan.gov/forms/1dfc6aea7e82b6f4ee25e29cc208e11ca89ef934eae992bb395e230b.pdf


error: ../data/raw/www.courts.michigan.gov/forms/ef741c7731613d12a69e7ae5d3e278c978bd75ddc40fa034946e014b.pdf


error: ../data/raw/www.courts.michigan.gov/forms/a74e0a8e48469686e643ff85ab21fd1e4cac0d3779a6984953341a41.pdf


error: ../data/raw/www.courts.michigan.gov/forms/a8c560095cf6932e3031acc49259d1dce809fe29e772fe3b35a80171.pdf
error: ../data/raw/www.courts.michigan.gov/forms/2a0847b00aa3dcf37dff8c1b2c3d4dcb3247445678cda13ff68553b7.pdf
error: ../data/raw/www.courts.michigan.gov/forms/ebf739852305fe98fe474c1f8e064bf3a8f58ee73cf55855c4ef172f.pdf


,id,jurisdiction,source,title,group,url,filename,downloaded,pages,fields,fields_conf,fields_old,f_per_p,reading
0,04b3a0734774c02edf8eb9056d23954aa38e96c77c3392...,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-11,2,"[name__1, name__2, *docket_number, *users1_add...","[0.9999998534627467, 0.9999998970227537, 1.0, ...","[NAME 1, NAME 2, CASE NUMBER, ADDRESS, DATE OF...",7.5,10
1,6e420f1b3575cfd8ef94b71977da9e38252e3395a78439...,UT,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-11,2,"[*docket_number, *users1_birthdate, male, juve...","[1.0, 1.0, 0.9999998196202323, 0.9999993742951...","[Court Case, Date of Birth, Male, Juveniles Na...",19.0,11
2,2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065...,UT,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-11,5,"[probation_department_court, special_condition...","[0.9999999495364337, 0.5418545963950205, 0.999...",[of the probation department or by order of th...,4.8,15
3,f6a6814890f21c11524d5785d772272916ff95909b7dba...,UT,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-11,3,"[*users1_name, *users1_address_line_one, *user...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.9999957319136206, ...","[Name, Address, City State Zip, Phone, Email, ...",14.666667,10
4,be656a9a361db7c3532026b0bae372e704beb160fce37e...,UT,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-11,1,"[*users1_name, *users1_address_line_one, *user...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.9999957319136206, ...","[Name, Address, City State Zip, Phone, Email, ...",19.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523,31f0385280ba8218dbace7693ea04977c1cfa3278ec4f8...,MI,www.courts.michigan.gov,Claim of Appeal of Right/Request to Withdraw A...,civil infraction,https://www.courts.michigan.gov/4a2ca8/siteass...,cia05.pdf,2021-11-11,1,"[disno, *docket_number, judge, *users1_address...","[0.9999993416564035, 1.0, 0.9999941855102024, ...","[disno, caseno, judge, address, telno, name, d...",40.0,20
1524,2a0847b00aa3dcf37dff8c1b2c3d4dcb3247445678cda1...,MI,www.courts.michigan.gov,"Default Judgment, Civil Infraction",civil infraction,https://www.courts.michigan.gov/4a7328/siteass...,cia07.pdf,2021-11-11,,,,,,
1525,07e5f95f3f4042ecd5b58fcf8a141e8931ab84124a6c19...,MI,www.courts.michigan.gov,"Judgment, Civil Infraction",civil infraction,https://www.courts.michigan.gov/4a7336/siteass...,cia02.pdf,2021-11-11,2,"[state_district_county, state, township, *user...","[0.9999999040213644, 0.9999997884967363, 0.999...","[STATE OF MICHIGAN JUDICIAL DISTRICT COUNTY, S...",20.5,9
1526,15f70625faa5b1a09b1839d4003d806921983fea9a8b65...,MI,www.courts.michigan.gov,Motion to Set Aside Default Judgment and Order...,civil infraction,https://www.courts.michigan.gov/4a2cb4/siteass...,cia04.pdf,2021-11-11,1,"[disno, *docket_number, judge, *users1_address...","[0.9999993416564035, 1.0, 0.9999941855102024, ...","[disno, caseno, judge, address, telno, dname, ...",22.0,17


In [11]:
df.to_csv("../data/processed/form_data.csv", index=False, encoding="utf-8")    

In [ ]:
#os.system('cp ../data/processed/www.utcourts.gov/forms/* ../data/processed/forms/')
#os.system('cp ../data/processed/www.courts.michigan.gov/forms/* ../data/processed/forms/')